In [1]:
pip install pdfplumber

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.0/49.0 kB 1.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 28.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.0/3.0 MB 64.4 MB/s eta 0:00:00


In [2]:
pip install pytesseract

In [3]:
!pip install tabula-py

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.0/12.0 MB 54.1 MB/s eta 0:00:00


In [4]:
pip install pymupdf

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.4/4.4 MB 9.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 30.6/30.6 MB 36.4 MB/s eta 0:00:00


In [5]:
pip install PyPDF2

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.6/232.6 kB 5.3 MB/s eta 0:00:00


In [6]:
pip install pdfminer.six

In [7]:
pip install pdf2image

In [8]:
pip install Pillow

In [9]:
!sudo apt install tesseract-ocr
!pip install pytesseract

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following additional packages will be installed:
  tesseract-ocr-eng tesseract-ocr-osd
The following NEW packages will be installed:
  tesseract-ocr tesseract-ocr-eng tesseract-ocr-osd
0 upgraded, 3 newly installed, 0 to remove and 24 not upgraded.
Need to get 4,816 kB of archives.
After this operation, 15.6 MB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu jammy/universe amd64 tesseract-ocr-eng all 1:4.00~git30-7274cfa-1.1 [1,591 kB]
Get:2 http://archive.ubuntu.com/ubuntu jammy/universe amd64 tesseract-ocr-osd all 1:4.00~git30-7274cfa-1.1 [2,990 kB]
Get:3 http://archive.ubuntu.com/ubuntu jammy/universe amd64 tesseract-ocr amd64 4.1.1-2.1build1 [236 kB]
Fetched 4,816 kB in 1s (5,055 kB/s)
debconf: unable to initialize frontend: Dialog
debconf: (No usable dialog-like program is installed, so the dialog based frontend cannot be used. at /usr/share/perl5/Debc

In [10]:
!apt-get install poppler-utils

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following NEW packages will be installed:
  poppler-utils
0 upgraded, 1 newly installed, 0 to remove and 24 not upgraded.
Need to get 186 kB of archives.
After this operation, 696 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu jammy-updates/main amd64 poppler-utils amd64 22.02.0-2ubuntu0.3 [186 kB]
Fetched 186 kB in 0s (384 kB/s)
Selecting previously unselected package poppler-utils.
(Reading database ... 121705 files and directories currently installed.)
Preparing to unpack .../poppler-utils_22.02.0-2ubuntu0.3_amd64.deb ...
Unpacking poppler-utils (22.02.0-2ubuntu0.3) ...
Setting up poppler-utils (22.02.0-2ubuntu0.3) ...
Processing triggers for man-db (2.10.2-1) ...


In [12]:
pip install unidecode

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 235.5/235.5 kB 4.5 MB/s eta 0:00:00


#Table Extraction Method 1

In [ ]:
from tabula.io import read_pdf
import tabula

In [ ]:
pdf_path = "9.pdf"

dfs = tabula.read_pdf(pdf_path, pages = "all", lattice=True)

tables = {}


for i in range(len(dfs)):
    # dfs[i].to_csv(f"table{i}.csv")
    d = {}
    print(dfs[i])
    tab = dfs[i]
    for col in tab.columns:
        d[col] = []
        for val in tab[col]:
            d[col].append(val)
    tables["table" + str(i+1)] = d

In [ ]:
tables

#Table Extraction Method 2

In [50]:
# To read the PDF
import PyPDF2
# To analyze the PDF layout and extract text
from pdfminer.high_level import extract_pages, extract_text
from pdfminer.layout import LTTextContainer, LTChar, LTRect, LTFigure
# To extract text from tables in PDF
import pdfplumber
# To extract the images from the PDFs
from PIL import Image
from pdf2image import convert_from_path
# To perform OCR to extract text from images
import pytesseract
# To remove the additional created files
import os

In [51]:
# Create a function to extract text

def text_extraction(element):
    # Extracting the text from the in line text element
    line_text = element.get_text()

    # Find the formats of the text
    # Initialize the list with all the formats appeared in the line of text
    line_formats = []
    for text_line in element:
        if isinstance(text_line, LTTextContainer):
            # Iterating through each character in the line of text
            for character in text_line:
                if isinstance(character, LTChar):
                    # Append the font name of the character
                    line_formats.append(character.fontname)
                    # Append the font size of the character
                    line_formats.append(character.size)
    # Find the unique font sizes and names in the line
    format_per_line = list(set(line_formats))

    # Return a tuple with the text in each line along with its format
    return (line_text, format_per_line)

In [52]:
# Extracting tables from the page

def extract_table(pdf_path, page_num, table_num):
    # Open the pdf file
    pdf = pdfplumber.open(pdf_path)
    # Find the examined page
    table_page = pdf.pages[page_num]
    # Extract the appropriate table
    table = table_page.extract_tables()[table_num]

    return table

# Convert table into appropriate fromat
def table_converter(table):
    table_string = ''
    # Iterate through each row of the table
    for row_num in range(len(table)):
        row = table[row_num]
        # Remove the line breaker from the wrapted texts
        cleaned_row = [item.replace('\n', ' ') if item is not None and '\n' in item else 'None' if item is None else item for item in row]
        # Convert the table into a string
        table_string+=('|'+'|'.join(cleaned_row)+'|'+'\n')
    # Removing the last line break
    table_string = table_string[:-1]
    return table_string

# Create a function to check if the element is in any tables present in the page
def is_element_inside_any_table(element, page ,tables):
    x0, y0up, x1, y1up = element.bbox
    # Change the cordinates because the pdfminer counts from the botton to top of the page
    y0 = page.bbox[3] - y1up
    y1 = page.bbox[3] - y0up
    for table in tables:
        tx0, ty0, tx1, ty1 = table.bbox
        if tx0 <= x0 <= x1 <= tx1 and ty0 <= y0 <= y1 <= ty1:
            return True
    return False

# Function to find the table for a given element
def find_table_for_element(element, page ,tables):
    x0, y0up, x1, y1up = element.bbox
    # Change the cordinates because the pdfminer counts from the botton to top of the page
    y0 = page.bbox[3] - y1up
    y1 = page.bbox[3] - y0up
    for i, table in enumerate(tables):
        tx0, ty0, tx1, ty1 = table.bbox
        if tx0 <= x0 <= x1 <= tx1 and ty0 <= y0 <= y1 <= ty1:
            return i  # Return the index of the table
    return None

In [55]:
# Create a function to crop the image elements from PDFs
def crop_image(element, pageObj):
    # Get the coordinates to crop the image from PDF
    [image_left, image_top, image_right, image_bottom] = [element.x0,element.y0,element.x1,element.y1]
    # Crop the page using coordinates (left, bottom, right, top)
    pageObj.mediabox.lower_left = (image_left, image_bottom)
    pageObj.mediabox.upper_right = (image_right, image_top)
    # Save the cropped page to a new PDF
    cropped_pdf_writer = PyPDF2.PdfWriter()
    cropped_pdf_writer.add_page(pageObj)
    # Save the cropped PDF to a new file
    with open('cropped_image.pdf', 'wb') as cropped_pdf_file:
        cropped_pdf_writer.write(cropped_pdf_file)

# Create a function to convert the PDF to images
def convert_to_images(input_file,):
    images = convert_from_path(input_file)
    image = images[0]
    output_file = 'PDF_image.png'
    image.save(output_file, 'PNG')

# Create a function to read text from images
def image_to_text(image_path):
    # Read the image
    img = Image.open(image_path)
    # Extract the text from the image
    text = pytesseract.image_to_string(img)
    return text

In [56]:
pdf_path = "1.pdf"
# Create a pdf file object
pdfFileObj = open(pdf_path, 'rb')
# Create a pdf reader object
pdfReaded = PyPDF2.PdfReader(pdfFileObj)

In [57]:
# Create the dictionary to extract text from each image
text_per_page = {}
# Create a boolean variable for image detection
image_flag = False

# We extract the pages from the PDF
for pagenum, page in enumerate(extract_pages(pdf_path)):

    # Initialize the variables needed for the text extraction from the page
    pageObj = pdfReaded.pages[pagenum]
    page_text = []
    line_format = []
    text_from_images = []
    text_from_tables = []
    page_content = []
    # Initialize the number of the examined tables
    table_in_page= -1
    # Open the pdf file
    pdf = pdfplumber.open(pdf_path)
    # Find the examined page
    page_tables = pdf.pages[pagenum]
    # Find the number of tables in the page
    tables = page_tables.find_tables()
    if len(tables)!=0:
        table_in_page = 0

    # Extracting the tables of the page
    for table_num in range(len(tables)):
        # Extract the information of the table
        table = extract_table(pdf_path, pagenum, table_num)
        # Convert the table information in structured string format
        table_string = table_converter(table)
        # Append the table string into a list
        text_from_tables.append(table_string)

    # Find all the elements
    page_elements = [(element.y1, element) for element in page._objs]
    # Sort all the element as they appear in the page
    page_elements.sort(key=lambda a: a[0], reverse=True)


    # Find the elements that composed a page
    for i,component in enumerate(page_elements):
        # Extract the element of the page layout
        element = component[1]

        # Check the elements for tables
        if table_in_page == -1:
            pass
        else:
            if is_element_inside_any_table(element, page ,tables):
                table_found = find_table_for_element(element,page ,tables)
                if table_found == table_in_page and table_found != None:
                    page_content.append(text_from_tables[table_in_page])
                    page_text.append('table')
                    line_format.append('table')
                    table_in_page+=1
                # Pass this iteration because the content of this element was extracted from the tables
                continue

        if not is_element_inside_any_table(element,page,tables):

            # Check if the element is text element
            if isinstance(element, LTTextContainer):
                # Use the function to extract the text and format for each text element
                (line_text, format_per_line) = text_extraction(element)
                # Append the text of each line to the page text
                page_text.append(line_text)
                # Append the format for each line containing text
                line_format.append(format_per_line)
                page_content.append(line_text)


            # Check the elements for images
            if isinstance(element, LTFigure):
                # Crop the image from PDF
                crop_image(element, pageObj)
                # Convert the croped pdf to image
                convert_to_images('cropped_image.pdf')
                # Extract the text from image
                image_text = image_to_text('PDF_image.png')
                text_from_images.append(image_text)
                page_content.append(image_text)
                # Add a placeholder in the text and format lists
                page_text.append('image')
                line_format.append('image')
                # Update the flag for image detection
                image_flag = True


    # Create the key of the dictionary
    dctkey = 'Page_'+str(pagenum)
    # Add the list of list as value of the page key
    text_per_page[dctkey]= [page_text, line_format, text_from_images,text_from_tables, page_content]

In [58]:
# Close the pdf file object
pdfFileObj.close()

In [59]:
if image_flag:
    os.remove('cropped_image.pdf')
    os.remove('PDF_image.png')

In [60]:
len(text_from_tables)

0

In [61]:
for i in range(len(text_per_page)):
    for tab in text_per_page["Page_" + str(i)][3]:
        print(tab)
        #type(tab)
        print("\n")

|Key Points|
|We evaluated the efficacy and safety of direct oral anti coagulants (DOACs) in patients who received a biopro thetic valve replacement.|
|In this cohort of patients, similar rates of thrombo- embolic complications and major bleeding events in patients receiving a DOAC or warfarin within 6 month of receiving a bioprosthetic valve replacement were se|
|Larger prospective trials are needed to evaluate the effi cacy and safety of DOACs in this population; however this study adds to the minimal literature available in th patient population.|


|Patients with valve replacement surgery AND received a DOAC or warfarin (n=238) Excluded: 41 patients Mechanical valve replacement (n=30) Discontinued anticoagulation before 6 months or switched agents (n=11) Included (n=197) Apixaban (n=86) Rivaroxaban (n=40) Dabigatran (n=1) Warfarin (n=70)|None|
|Apixaban (n=86)|Warfarin (n=70)|




In [62]:
import pandas as pd
import numpy as np

In [63]:
tables = []
for i in range(len(text_per_page)):
    for tab in text_per_page["Page_" + str(i)][3]:
        r = tab.split('\n')
        lst = []
        for rows in r:
            lst.append(rows.split('|'))
        print(pd.DataFrame(lst))
        tables.append(pd.DataFrame(lst))
        print("\n")

  0                                                  1 2
0                                           Key Points  
1    We evaluated the efficacy and safety of direct...  
2    In this cohort of patients, similar rates of t...  
3    Larger prospective trials are needed to evalua...  


  0                                                  1                2 3
0    Patients with valve replacement surgery AND re...             None  
1                                      Apixaban (n=86)  Warfarin (n=70)  




In [64]:
idx = 0
unwanted = []
for table in tables:
    r = []
    for index, row in table.iterrows():
        for ele in row:
            if ele == 'None':
                r.append(index)
                break
    table.drop(r, inplace=True)

    c = []
    for col in table.columns:
        cnt = 0
        for ele in table[col]:
            if ele == '':
                cnt += 1
        if(len(table) == cnt):
            c.append(col)

    table.drop(c, axis=1, inplace=True)

    if len(table) <= 3:
        unwanted.append(idx)

    idx += 1

In [65]:
tables

[                                                   1
 0                                         Key Points
 1  We evaluated the efficacy and safety of direct...
 2  In this cohort of patients, similar rates of t...
 3  Larger prospective trials are needed to evalua...,
                  1                2
 1  Apixaban (n=86)  Warfarin (n=70)]

In [66]:
final_tables = []

for i in range(len(tables)):
    if i in unwanted:
        continue
    final_tables.append(tables[i])

In [67]:
final_tables

[                                                   1
 0                                         Key Points
 1  We evaluated the efficacy and safety of direct...
 2  In this cohort of patients, similar rates of t...
 3  Larger prospective trials are needed to evalua...]

In [68]:
t = {}


for i in range(len(final_tables)):
    # dfs[i].to_csv(f"table{i}.csv")
    d = {}
    print(final_tables[i])
    tab = final_tables[i]
    for col in tab.columns:
        d[col] = []
        for val in tab[col]:
            d[col].append(val)
    t["table" + str(i+1)] = d

t

                                                   1
0                                         Key Points
1  We evaluated the efficacy and safety of direct...
2  In this cohort of patients, similar rates of t...
3  Larger prospective trials are needed to evalua...


{'table1': {1: ['Key Points',
   'We evaluated the efficacy and safety of direct oral anti coagulants (DOACs) in patients who received a biopro thetic valve replacement.',
   'In this cohort of patients, similar rates of thrombo- embolic complications and major bleeding events in patients receiving a DOAC or warfarin within 6 month of receiving a bioprosthetic valve replacement were se',
   'Larger prospective trials are needed to evaluate the effi cacy and safety of DOACs in this population; however this study adds to the minimal literature available in th patient population.']}}

#Extracting Text

In [69]:
import fitz

my_path = "1.pdf"
doc = fitz.open(my_path)

for page in doc:
    text = page.get_text()
    print(text)

Vol.:(0123456789)
Clinical Drug Investigation 
https://doi.org/10.1007/s40261-020-00939-x
ORIGINAL RESEARCH ARTICLE
Comparison of the Efficacy and Safety of Direct Oral Anticoagulants 
and Warfarin After Bioprosthetic Valve Replacements
Stacy Pasciolla1,2  · Laura Falconieri Zizza1 · Thuy Le1 · Kesha Wright1
 
© Springer Nature Switzerland AG 2020
Abstract
Background and Objective Current guidelines recommend anticoagulation with a vitamin K antagonist (warfarin) after a 
bioprosthetic valve replacement. There is minimal literature evaluating direct oral anticoagulants (DOACs) in patients who 
have just received a bioprosthetic aortic valve replacement (AVR) or mitral valve replacement (MVR). The purpose of this 
study was to investigate any differences in efficacy and safety for patients taking a DOAC, compared with warfarin, after a 
bioprosthetic AVR or MVR.
Methods A retrospective cohort study was performed to evaluate anticoagulation in patients who received bioprosthetic 
valve r

In [70]:
output = page.get_text("blocks")

In [71]:
for item in output:
    print(item)

(51.02360153198242, 31.765121459960938, 312.85760498046875, 41.33612060546875, 'Efficacy and Safety of DOACs Vs. Warfarin After Bioprosthetic Valve Replacements\n', 0, 0)
(51.02360153198242, 535.5684204101562, 122.71162414550781, 550.4364013671875, '5  Conclusion\n', 1, 0)
(51.02360153198242, 562.2064208984375, 291.6336364746094, 639.79638671875, 'This small, exploratory study found similar rates of throm-\nboembolic complications and major bleeding events in \npatients receiving a DOAC or warfarin within 6\xa0months of \nreceiving a bioprosthetic MVR or AVR. Larger, prospec-\ntive trials are needed to evaluate the efficacy and safety of \nDOACs in this population.\n', 2, 0)
(51.02360153198242, 660.5684204101562, 109.8716049194336, 675.4364013671875, 'References\n', 3, 0)
(51.02360153198242, 686.2409057617188, 291.32977294921875, 719.0593872070312, ' 1. Nishimura RA, Otto CM, Bonow RO, et\xa0al. 2017 AHA/ACC \nFocused Update of the 2014 AHA/ACC Guideline for the man-\nagement of patien

In [72]:
for page in doc:
    output = page.get_text("blocks")
    previous_block_id = 0 # Set a variable to mark the block id
    for block in output:
        if block[6] == 0: # We only take the text
            if previous_block_id != block[5]:
                # Compare the block number
                previous_block_id = block[5]
                print("\n")
            print(block[4])



Vol.:(0123456789)



Clinical Drug Investigation 
https://doi.org/10.1007/s40261-020-00939-x



ORIGINAL RESEARCH ARTICLE



Comparison of the Efficacy and Safety of Direct Oral Anticoagulants 
and Warfarin After Bioprosthetic Valve Replacements



Stacy Pasciolla1,2  · Laura Falconieri Zizza1 · Thuy Le1 · Kesha Wright1



 
© Springer Nature Switzerland AG 2020



Abstract
Background and Objective Current guidelines recommend anticoagulation with a vitamin K antagonist (warfarin) after a 
bioprosthetic valve replacement. There is minimal literature evaluating direct oral anticoagulants (DOACs) in patients who 
have just received a bioprosthetic aortic valve replacement (AVR) or mitral valve replacement (MVR). The purpose of this 
study was to investigate any differences in efficacy and safety for patients taking a DOAC, compared with warfarin, after a 
bioprosthetic AVR or MVR.
Methods A retrospective cohort study was performed to evaluate anticoagulation in patients who received bi

In [73]:
from unidecode import unidecode

output = []
for page in doc:
    output += page.get_text("blocks")
previous_block_id = 0 # Set a variable to mark the block id
for block in output:
    if block[6] == 0: # We only take the text
        if previous_block_id != block[5]: # Compare the block number
            print("\n")
            plain_text = unidecode(block[4])
            print(plain_text)



Vol.:(0123456789)



Clinical Drug Investigation 
https://doi.org/10.1007/s40261-020-00939-x



ORIGINAL RESEARCH ARTICLE



Comparison of the Efficacy and Safety of Direct Oral Anticoagulants 
and Warfarin After Bioprosthetic Valve Replacements



Stacy Pasciolla1,2  * Laura Falconieri Zizza1 * Thuy Le1 * Kesha Wright1



 
(c) Springer Nature Switzerland AG 2020



Abstract
Background and Objective Current guidelines recommend anticoagulation with a vitamin K antagonist (warfarin) after a 
bioprosthetic valve replacement. There is minimal literature evaluating direct oral anticoagulants (DOACs) in patients who 
have just received a bioprosthetic aortic valve replacement (AVR) or mitral valve replacement (MVR). The purpose of this 
study was to investigate any differences in efficacy and safety for patients taking a DOAC, compared with warfarin, after a 
bioprosthetic AVR or MVR.
Methods A retrospective cohort study was performed to evaluate anticoagulation in patients who received 

In [74]:
output[0]

(51.02362060546875,
 737.0077514648438,
 183.02362060546875,
 747.8077392578125,
 '<image: DeviceGray, width: 1100, height: 90, bpc: 1>',
 0,
 1)

In [75]:
block_dict = {}
page_num = 1
for page in doc: # Iterate all pages in the document
    file_dict = page.get_text('dict') # Get the page dictionary
    block = file_dict['blocks'] # Get the block information
    block_dict[page_num] = block # Store in block dictionary
    page_num += 1 # Increase the page value by 1

In [76]:
import re
spans = pd.DataFrame(columns=['xmin', 'ymin', 'xmax', 'ymax', 'text', 'tag'])
rows = []
for page_num, blocks in block_dict.items():
    for block in blocks:
        if block['type'] == 0:
            for line in block['lines']:
                for span in line['spans']:
                    xmin, ymin, xmax, ymax = list(span['bbox'])
                    font_size = span['size']
                    text = unidecode(span['text'])
                    span_font = span['font']
                    is_upper = False
                    is_bold = False
                    if "bold" in span_font.lower():
                        is_bold = True
                    if re.sub("[\(\[].*?[\)\]]", "", text).isupper():
                        is_upper = True
                    if text.replace(" ","") !=  "":
                        rows.append((xmin, ymin, xmax, ymax, text, is_upper, is_bold, span_font, font_size))
span_df = pd.DataFrame(rows, columns=['xmin','ymin','xmax','ymax', 'text', 'is_upper','is_bold','span_font', 'font_size'])

In [77]:
span_df

,xmin,ymin,xmax,ymax,text,is_upper,is_bold,span_font,font_size
0,485.480408,737.972046,540.672302,746.980042,Vol.:(0123456789),False,False,MyriadPro-SemiCn2,8.0
1,51.023602,32.365158,138.300781,41.936157,Clinical Drug Investigation,False,False,MyriadPro-SemiCn,8.5
2,51.023602,43.695663,196.824188,53.266663,https://doi.org/10.1007/s40261-020-00939-x,False,False,MyriadPro-SemiCn,8.5
3,56.787899,64.210640,194.574005,75.819641,ORIGINAL RESEARCH ARTICLE,True,True,MyriadPro-Semibold,9.5
4,51.023602,103.174950,483.673157,122.582947,Comparison of the Efficacy and Safety of Direc...,False,True,MyriadPro-SemiboldSemiCn,16.0
...,...,...,...,...,...,...,...,...,...
1113,157.340454,401.012909,173.626465,413.839417,1 (2),False,False,STIX-Regular,8.5
1114,229.981445,401.012909,234.231445,413.839417,0,False,False,STIX-Regular,8.5
1115,51.124451,413.541901,70.546951,426.368408,Other,False,False,STIX-Regular,8.5
1116,157.348938,413.541901,184.259949,426.368408,7 (14.3),False,False,STIX-Regular,8.5


In [78]:
span_scores = []
span_num_occur = {}
special = '[(_:/,#%\=@)]'

for index, span_row in span_df.iterrows():

    score = round(span_row.font_size)
    text = span_row.text

    if not re.search(special, text):
        if span_row.is_bold:
            score +=1

        if span_row.is_upper:
            score +=1
    span_scores.append(score)

values, counts = np.unique(span_scores, return_counts=True)

In [79]:
values, counts

(array([ 6,  7,  8,  9, 10, 11, 12, 13, 17]),
 array([ 11,   6, 604,  40, 434,  12,   3,   6,   2]))

In [80]:
values, counts = np.unique(span_scores, return_counts=True)

style_dict = {}

for value, count in zip(values, counts):
    style_dict[value] = count

sorted(style_dict.items(), key=lambda x: x[1])

[(17, 2),
 (12, 3),
 (7, 6),
 (13, 6),
 (6, 11),
 (11, 12),
 (9, 40),
 (10, 434),
 (8, 604)]

In [81]:
p_size = max(style_dict, key=style_dict.get)
idx = 0
tag = {}

for size in sorted(values, reverse = True):

    idx += 1
    if size == p_size:
        idx = 0
        tag[size] = 'p'

    if size > p_size:
        tag[size] = 'h{0}'.format(idx)

    if size < p_size:
        tag[size] = 's{0}'.format(idx)

In [82]:
p_size

8

In [83]:
tag

{17: 'h1',
 13: 'h2',
 12: 'h3',
 11: 'h4',
 10: 'h5',
 9: 'h6',
 8: 'p',
 7: 's1',
 6: 's2'}

In [84]:
span_tags = [tag[score] for score in span_scores]

span_df['tag'] = span_tags

In [85]:
span_df

,xmin,ymin,xmax,ymax,text,is_upper,is_bold,span_font,font_size,tag
0,485.480408,737.972046,540.672302,746.980042,Vol.:(0123456789),False,False,MyriadPro-SemiCn2,8.0,p
1,51.023602,32.365158,138.300781,41.936157,Clinical Drug Investigation,False,False,MyriadPro-SemiCn,8.5,p
2,51.023602,43.695663,196.824188,53.266663,https://doi.org/10.1007/s40261-020-00939-x,False,False,MyriadPro-SemiCn,8.5,p
3,56.787899,64.210640,194.574005,75.819641,ORIGINAL RESEARCH ARTICLE,True,True,MyriadPro-Semibold,9.5,h3
4,51.023602,103.174950,483.673157,122.582947,Comparison of the Efficacy and Safety of Direc...,False,True,MyriadPro-SemiboldSemiCn,16.0,h1
...,...,...,...,...,...,...,...,...,...,...
1113,157.340454,401.012909,173.626465,413.839417,1 (2),False,False,STIX-Regular,8.5,p
1114,229.981445,401.012909,234.231445,413.839417,0,False,False,STIX-Regular,8.5,p
1115,51.124451,413.541901,70.546951,426.368408,Other,False,False,STIX-Regular,8.5,p
1116,157.348938,413.541901,184.259949,426.368408,7 (14.3),False,False,STIX-Regular,8.5,p


In [86]:
headings_list = []
text_list = []
tmp = []

heading = ''

for index, span_row in span_df.iterrows():

    text = span_row.text
    tag = span_row.tag

    if 'h' in tag and span_row.is_bold == True:
        headings_list.append(text)
        text_list.append('\n'.join(tmp))
        tmp = []
        heading = text

    else:
        tmp.append(text)

text_list.append('\n'.join(tmp))
text_list = text_list[1:]
text_df = pd.DataFrame(zip(headings_list, text_list),columns=['heading', 'content'] )

In [87]:
text_df

,heading,content
0,ORIGINAL RESEARCH ARTICLE,
1,Comparison of the Efficacy and Safety of Direc...,
2,and Warfarin After Bioprosthetic Valve Replace...,
3,Stacy Pasciolla,"1,2"
4,* Laura Falconieri Zizza,1
5,* Thuy Le,1
6,* Kesha Wright,1\n(c) Springer Nature Switzerland AG 2020
7,Abstract,
8,Background and Objective,Current guidelines recommend anticoagulation ...
9,Methods,A retrospective cohort study was performed to...


In [88]:
title = ""
for index, text_row in text_df.iterrows():
    if index < 5 and text_row["content"] == '':
        title += text_row["heading"]

In [89]:
title

'ORIGINAL RESEARCH ARTICLEComparison of the Efficacy and Safety of Direct Oral Anticoagulants and Warfarin After Bioprosthetic Valve Replacements'

In [90]:
useless = []

for index, text_row in text_df.iterrows():
    if "Table" in text_row["heading"] or "Fig" in text_row["heading"]:
        useless.append(index)

    if index >= 3 and text_row["content"] == '':
        useless.append(index)

text_df.drop(useless, inplace=True)

In [91]:
text_df

,heading,content
0,ORIGINAL RESEARCH ARTICLE,
1,Comparison of the Efficacy and Safety of Direc...,
2,and Warfarin After Bioprosthetic Valve Replace...,
3,Stacy Pasciolla,"1,2"
4,* Laura Falconieri Zizza,1
5,* Thuy Le,1
6,* Kesha Wright,1\n(c) Springer Nature Switzerland AG 2020
8,Background and Objective,Current guidelines recommend anticoagulation ...
9,Methods,A retrospective cohort study was performed to...
10,Results,A total of 197 patients were included; 70 pat...


In [92]:
response = {}
response["title"] = title

for index, text_row in text_df.iterrows():
    if index != 0 and text_row["content"] != '':
        response[text_row.heading] = text_row.content

In [97]:
response["tables"] = t

In [98]:
import json

json_string = json.dumps(response, indent=4)
print("response:")
print(json_string)

response:
{
    "title": "ORIGINAL RESEARCH ARTICLEComparison of the Efficacy and Safety of Direct Oral Anticoagulants and Warfarin After Bioprosthetic Valve Replacements",
    "Stacy Pasciolla": "1,2",
    "  * Laura Falconieri Zizza": "1",
    " * Thuy Le": "1",
    " * Kesha Wright": "1\n(c) Springer Nature Switzerland AG 2020",
    "Background and Objective": " Current guidelines recommend anticoagulation with a vitamin K antagonist (warfarin) after a \nbioprosthetic valve replacement. There is minimal literature evaluating direct oral anticoagulants (DOACs) in patients who \nhave just received a bioprosthetic aortic valve replacement (AVR) or mitral valve replacement (MVR). The purpose of this \nstudy was to investigate any differences in efficacy and safety for patients taking a DOAC, compared with warfarin, after a \nbioprosthetic AVR or MVR.",
    "Methods": " A retrospective cohort study was performed to evaluate anticoagulation in patients who received bioprosthetic \nvalve r

In [100]:
with open("output.json", "w") as outfile:
    outfile.write(json_string)